In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from fredapi import Fredw

In [ ]:
# Step 1: Define spread adjustments and tenors
spread_adjustments = {
    "ON": 0.00644 / 100, "1W": 0.03839 / 100, "1M": 0.11448 / 100, 
    "2M": 0.18456 / 100, "3M": 0.26161 / 100, "6M": 0.42826 / 100, 
    "12M": 0.71513 / 100
}
tenor_labels = ["ON", "1W", "1M", "2M", "3M", "6M", "12M"]
tenors = [1, 7, 30, 60, 90, 180, 365]  # Tenors in days

# Step 2: Set rate record date
rate_record_date = datetime(2021, 2, 26)
arr_day_count = 360

# Step 3: Fetch SOFR and Federal Funds Rate data from FRED
api_key = 'key'  # Replace FRED API key
fred = Fred(api_key=api_key)

# Fetch SOFR data from FRED
sofr_series = fred.get_series('SOFR', start_date=rate_record_date, end_date=datetime.today())

# Fetch Federal Funds Rate data from FRED
fed_funds_series = fred.get_series('FEDFUNDS', start_date=rate_record_date, end_date=datetime.today())

# Convert to DataFrame
sofr_data = pd.DataFrame(sofr_series, columns=["SOFR"])
fed_funds_data = pd.DataFrame(fed_funds_series, columns=["FEDFUNDS"])

# Merge the data on the date index
data = pd.merge(sofr_data, fed_funds_data, left_index=True, right_index=True, how="inner")
data.index.name = "Date"
data = data.reset_index()

# Ensure data is sorted by Date (if not already)
data = data.sort_values(by="Date")

# Step 4: Define ARR and Fallback Rate Calculation functions
def calculate_arr(sofr_data, rate_record_date, calculation_date, arr_day_count=360):
    # Filter SOFR data for the period between rate_record_date and calculation_date
    sofr_subset = sofr_data[(sofr_data["Date"] >= rate_record_date) & (sofr_data["Date"] <= calculation_date)]
    
    if len(sofr_subset) > 1:
        tau = np.diff(sofr_subset["Date"]).astype('timedelta64[D]').astype(int) / arr_day_count
        rel_rate = sofr_subset["SOFR"].iloc[:-1].values

        # Compound the rates
        comp_rate = np.prod(1 + tau * rel_rate / 100) - 1

        # Compute ARR
        arr = (arr_day_count / (calculation_date - rate_record_date).days) * comp_rate
        return round(arr, 7)
    else:
        return 0  # If not enough data

def calculate_spread_adjustment(ibor_data, arr_data, median_period):
    # Calculate the median of (IBOR value - ARR value)
    differences = ibor_data - arr_data
    return np.median(differences)

def calculate_fallback_rate(arr, spread_adj):
    return arr + spread_adj

# Step 5: Loop through all tenors and compute the fallback rate for each
fallback_rates = []

for rate_tenor in tenor_labels:
    # Step 5a: Calculate calculation date for each tenor
    calculation_date = rate_record_date + timedelta(days=tenors[tenor_labels.index(rate_tenor)])

    # Step 5b: Adjust calculation date to the nearest business day if needed (for simplicity, assume it is)
    # For now, using SOFR data as reference for ARR calculation
    arr = calculate_arr(data, rate_record_date, calculation_date)
    spread_adj = spread_adjustments[rate_tenor]  # Example spread adjustment for each tenor

    # Calculate Fallback Rate
    fallback_rate = calculate_fallback_rate(arr, spread_adj)

    # Add the result to the list
    fallback_rates.append([rate_tenor, spread_adj, arr, fallback_rate])

# Step 6: Display the results in a table
columns = ["Tenor", "Spread Adjustment", "ARR", "Fallback Rate"]
fallback_table = pd.DataFrame(fallback_rates, columns=columns)

# Print the results
fallback_table


,Tenor,Spread Adjustment,ARR,Fallback Rate
0,ON,0.000064,0.000000,0.000064
1,1W,0.000384,0.000000,0.000384
2,1M,0.001145,0.000000,0.001145
3,2M,0.001846,0.000103,0.001949
4,3M,0.002616,0.000069,0.002685
5,6M,0.004283,0.000085,0.004368
6,12M,0.007151,0.000336,0.007488
